In [1]:
def simulator():
    for file in os.listdir("./pickles"):
        if file.find("simulator"):
            os.remove("pickles/"+file)
    
    
    # build a list of 3 random models
    Simulator_Model = namedtuple('Simulator_Model', ['id', 'T', 'D'])
    models = []
    for j in range(3):
        A = np.random.randn(4, 4)
        eigvals1 = linalg.eigvals(A)
        lambda_ = np.max(np.abs(eigvals1))
        delta_ = np.random.uniform(1.01,1.1)
        A = A/(delta_*lambda_)
        
        mu = np.array([[np.random.uniform(80,100)],[np.random.uniform(90,110)],[np.random.uniform(20,35)],[np.random.uniform(20,35)]])
        
        model = Simulator_Model(j, A, mu)
        models.append(model)
    
    
    # simulate a data for 3 periods (each period based on a randomly chosen model from the list)
    period_length_array = np.arange(30,50) # period_length/10
    
    s0 = np.random.randn(4, 1)
    s_n = s0
    x_n = s_n + mu
    
    hr = []
    sys_bp = []
    rr = []
    dia_bp = []
    hr.append(x_n[0])
    sys_bp.append(x_n[1])
    rr.append(x_n[2])
    dia_bp.append(x_n[3])
    total_length = 0
    borders = []
    model = Simulator_Model(-1, None, None)
    for j in range(3):
        pre_model = model
        model = random.choice(models)
        length = random.choice(period_length_array)
        total_length = total_length + length
        if pre_model.id != model.id and j != 0:
            borders.append(total_length-length)
            
        print("model " + str(model.id) + " from " + str((int)((total_length - length)/F)) + " to " + str((int)(total_length/F)))
        print("T = \n", model.T)
        print("mu = \n", model.D)
        print("")
        
        for i in range(length):
            s_n = np.dot(model.T, s_n) + np.random.randn(4, 1)
            x_n = s_n + model.D
            hr.append(x_n[0])
            sys_bp.append(x_n[1])
            rr.append(x_n[2])
            dia_bp.append(x_n[3])
    data = np.row_stack((np.array(hr).transpose(),np.array(sys_bp).transpose(),np.array(rr).transpose(),np.array(dia_bp).transpose()))
    hr = data[0]
    sys_bp = data[1]
    rr = data[2]
    dia_bp = data[3]
    time = np.arange(0,(int)(len(hr)/F),(int)(1/F))

    
    # plot the simulated data
    fig, ax = plt.subplots(nrows=4, ncols=1, figsize=(24, 22))
    
    ax[0].set_title('Simulated Heart Rate')
    ax[0].set_xlabel('Time [sec]')
    ax[0].set_ylabel('[bpm]')
    ax[0].plot(time, hr)    

    ax[1].set_title('Simulated Systolic Blood Pressure')
    ax[1].set_xlabel('Time [sec]')
    ax[1].set_ylabel('[mmHg]')
    ax[1].plot(time, sys_bp)

    ax[2].set_title('Simulated Respiration Rate')
    ax[2].set_xlabel('Time [sec]')
    ax[2].set_ylabel('[Hz]')
    ax[2].plot(time, rr)
    
    ax[3].set_title('Simulated Diastolic Blood Pressure')
    ax[3].set_xlabel('Time [sec]')
    ax[3].set_ylabel('[mmHg]')
    ax[3].plot(time, dia_bp)
    
    for i in [0,1,2,3]:
        for border in borders:
            ax[i].axvline(x=border/F, color='C1')
    
    plt.show()
    
    
    # save the simulated data in files
    with open('samples/simulated_processed_hr', 'wb') as file:
        pickle.dump(hr, file)
    with open('samples/simulated_processed_sys_bp', 'wb') as file:
        pickle.dump(sys_bp, file)
    with open('samples/simulated_processed_rr', 'wb') as file:
        pickle.dump(rr, file)
    with open('samples/simulated_processed_dia_bp', 'wb') as file:
        pickle.dump(dia_bp, file)
    with open('samples/simulated_time', 'wb') as file:
        pickle.dump(time, file)